In [68]:
import pandas as pd
import numpy as np
import re

# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql



In [6]:
# set up arguments needed to connect to AWS through psycopg2 and connect
connection_args = {
    'host': '3.14.5.169',  
    'user': 'ubuntu',
    'dbname': 'cpdp',  
    'port': 5432          
}
connection = pg.connect(**connection_args)

from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

## SQL Queries and Basic Division of Data

In [ ]:
#Import outcomes of all complaints
query = "SELECT * FROM outcomes;"
outcomes_df = pd_sql.read_sql(query, connection)
outcomes_df.to_pickle('outcomes_df')

In [7]:
outcomes_df = pd.read_pickle('outcomes_df')

In [17]:
# Isolate the category of complaint type (indicated by number)
outcomes_df['number_complaint_code']=outcomes_df['complaint_code'].str.extract("([0-9]+)")

# Drop the recommendation columns (only apply to recent complaints)
outcomes_nr =outcomes_df.drop(['recc_finding','recc_outcome'],axis=1)
outcomes_nr['officer_filed']=outcomes_nr['cr_id'].isin(comp23_off.cr_id)

#Divide up outcomes data by 3 periods of police reform
outcomes_cv1 = outcomes_nr[outcomes_nr['cv']==1]
outcomes_cv23 = outcomes_nr[(outcomes_nr['cv']==3)|(outcomes_nr['cv']==2)]

In [ ]:
# Import complaints by everyone from cv 1
query = "SELECT * FROM comp_cv1;"
comp1_df = pd_sql.read_sql(query, connection)
comp1_df.to_pickle('comp1_df')

# Drop complaints where little detail about complainants exists (not a high #)
comp1_df.dropna(inplace=True)
# Get mode of race, gender and birthday for complaints where there were multiple complainants
comp1_modes = comp1_df.groupby('cr_id').agg(lambda x:x.value_counts().index[0] )

# Join demographic complainant data from CV1 with the outcomes df
comp1_out = comp1_modes.join(outcomes_cv1.set_index('cr_id'),on='cr_id',how='right')

# For complaints where no complainant data was provided, fill in 'Unknown'
comp1_out.fillna('Unknown',inplace=True)

In [ ]:
# Import complaint witnesses by everyone from cv 1
query = "SELECT * FROM wit_cv1_all;"
wit1_df = pd_sql.read_sql(query, connection)

In [ ]:
# Import roster of police officers
query = "SELECT * FROM officers;"
officers = pd_sql.read_sql(query, connection)

In [ ]:
# Import awards list
query = "SELECT * FROM awards;"
awards= pd_sql.read_sql(query, connection)

In [15]:
# Import complaints by civilians from cv 2/3
query = "SELECT * FROM comp_cv23_civ;"
comp23_civ = pd_sql.read_sql(query, connection)
comp23_civ.to_pickle('comp23_civ')

In [16]:
# Import complaint details from all eras
query = "SELECT * FROM complaints;"
comp_det = pd_sql.read_sql(query, connection)

In [14]:
# Import officer witnesses from cv 2/3
query = "SELECT * FROM wit_cv23_off;"
wit23_off = pd_sql.read_sql(query2, connection)


In [13]:
# Import civilian witness from cv 2
query2 = "SELECT * FROM wit_cv2_civ;"
wit2_civ = pd_sql.read_sql(query2, connection)

# Import civilian witness from cv 3
query3 = "SELECT * FROM wit_cv3_civ;"
wit3_civ = pd_sql.read_sql(query3, connection)

wit_cv23=pd.concat([wit2_civ,wit3_civ],axis=0)

/Users/ivy/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [12]:
# Import complaints by officers from cv 2/3
query = "SELECT * FROM comp_cv23_off;"
comp23_off = pd_sql.read_sql(query, connection)

## Cleaning up the complaint data from 1967-1999 (CV1)

In [19]:
# Divide up the detailed complaint info amongst data collection period
comp_det_sm =comp_det.drop(['add1','add2','city','full_address','location'],axis=1)
compdet_cv1 = comp_det_sm[comp_det_sm['cv']==1]
compdet_cv23 = comp_det_sm[(comp_det_sm['cv']==2)|(comp_det_sm['cv']==3)]

In [ ]:
# Convert date details about complaints into datetime features
compdet_cv1['incident_dt']=pd.to_datetime(compdet_cv1['incident_date'])
compdet_cv1['complaint_dt']=pd.to_datetime(compdet_cv1['complaint_date'])
compdet_cv1['closed_dt']=pd.to_datetime(compdet_cv1['closed_date'])

In [ ]:
# Engineer features that indicate time between complaint and incident, and duration of investigation
compdet_cv1['time2complain']=(compdet_cv1.complaint_dt-compdet_cv1.incident_dt)
compdet_cv1['time2close']=(compdet_cv1.closed_dt-compdet_cv1.complaint_dt)

In [ ]:
#Make a smaller DF with most relevant columns and pickle it
compdet_cv1 = compdet_cv1[['cr_id','cv','beat','incident_dt','complaint_dt','closed_dt','time2complain','time2close']]
compdet_cv1.drop('beat',axis=1,inplace=True)
compdet_cv1.to_pickle('compdet_cv1_df')

### Counting the number of witnesses for each complaint, and noting whether officers were complainants

In [ ]:
# Count the number of witnesses for each complaint, and join 
# it with the total dataframe for CV1
numwit1 = pd.DataFrame(wit1_df.cr_id.value_counts())
numwit1.columns=['numwitnesses']
numwit1.index.name='cr_id'
comp1_out_wit = comp1_out.join(numwit1,on='cr_id')

# Replace NA values with zeros since there were no recorded witnesses
comp1_out_wit.numwitnesses.fillna(0,inplace=True)
# Convert records with no identified complainants as "unknown" - could be officers,
# could be bad record-keeping
comp1_out_wit.officer_filed[comp1_out_wit['officer_or_non'].isna()]='Unknown'
comp1_out_wit.to_pickle('comp1_partmerge')

In [ ]:
# Isolate the category of complaint type (indicated by number)
comp1_out_wit['number_complaint_code']=comp1_out_wit['complaint_code'].str.extract("([0-9]+)")

In [ ]:
# Join the df with complainant, outcome, and #witnesses with details about time of complaint
comp1_fulldet = compdet_cv1.join(comp1_out_wit.set_index('cr_id'),on='cr_id',how='left')
comp1_fulldet.dropna(inplace=True)

# Change 'officer-filed' status to 'unknown' if no complainant data was provided
comp1_fulldet['officer_filed'][comp1_fulldet['officer_or_non']=='Unknown']='Unknown'

## Getting information about officers

In [ ]:
# Drop unnecessary personal information about officers
officers_an = officers.drop(['uid','first_name','last_name','middle_initial','middle_initial2','suffix_name','current_rank','profile_count','current_star'],axis=1)

# Convert date fields to datetime fields
officers_an['birth_year']=pd.to_datetime(officers_an['birth_year'],format='%Y')
officers_an['appointed_date']=pd.to_datetime(officers_an['appointed_date'])
officers_an['resignation_date']=pd.to_datetime(officers_an['resignation_date'])
officers_an['start_date']=pd.to_datetime(officers_an['start_date'])
officers_an['org_hire_date']=pd.to_datetime(officers_an['org_hire_date'])

### Counting awards received by officers and adding those to officer roster

In [ ]:
# Slim down awards information to the basics
awards_id = awards[['award_type','award_request_date','link_uid']]

# Make a dataframe that has the # of awards won by each officer (IDd by link_uid)
officer_id = pd.DataFrame(awards_id.link_uid.value_counts()).rename(columns={'link_uid': 'num_awards'})
officer_id.index.rename('link_uid',inplace=True)


# Join the officers df with the awardsinfo, and fill in zero awards if none were listed
off_full = officer_id.join(officers_an.set_index('link_uid'),on='link_uid',how='right')
off_full.num_awards.fillna(0,inplace=True)

# Give columns unique names to avoid conflict with complainant headers 
off_full.rename(columns={"birth_year":'officer_birthyr','race':'officer_race','gender':'officer_gender'},inplace=True)
off_full.to_pickle('officers_awards')

### Counting total complaints received by officers

In [ ]:
# Create a dataframe that totals the complaints for each officer
num_complaints = pd.DataFrame(outcomes.link_uid.value_counts())
num_complaints.rename(columns={'link_uid':'num_complaints'},inplace=True)
num_complaints.index.rename('link_uid',inplace=True)

### Subgrouping complaints into types and counting how many received by each officers (note - some types of complaints were not described by CPD in FOIA, so not all complaints will be captured by this subgrouping)

In [ ]:
# Isolate the complaint code number (reveals broad category)
outcomes['number_complaint_code']=outcomes['complaint_code'].str.extract("([0-9]+)")

# Create masks for each of the categories of complaints, sorted into broader categories
mask_operations = (outcomes['number_complaint_code']=='17') | (outcomes['number_complaint_code']=='12') | (outcomes['number_complaint_code']=='10')|(outcomes['number_complaint_code']=='07')
mask_substance = (outcomes['number_complaint_code']=='02')| (outcomes['number_complaint_code']=='15')
mask_civil = ((outcomes['number_complaint_code']=='03')|(outcomes['number_complaint_code']=='04')|(outcomes['number_complaint_code']=='16'))
mask_abuse= (outcomes['number_complaint_code']=='05')|(outcomes['number_complaint_code']=='18')|(outcomes['number_complaint_code']=='19')|(outcomes['number_complaint_code']=='01')
mask_corrupt = (outcomes['number_complaint_code']=='20')|(outcomes['number_complaint_code']=='21')|(outcomes['number_complaint_code']=='06')|(outcomes['number_complaint_code']=='08')|(outcomes['number_complaint_code']=='09')

# Count the number of each type of complaint associated with each unique police officer, and rename the columns and indices
oper_num = pd.DataFrame(outcomes[mask_operations].link_uid.value_counts())
oper_num.rename(columns={'link_uid':'operations_ct'},inplace=True)
oper_num.index.rename('link_uid',inplace=True)

subs_num = pd.DataFrame(outcomes[mask_substance].link_uid.value_counts())
subs_num.rename(columns={'link_uid':'substance_ct'},inplace=True)
subs_num.index.rename('link_uid',inplace=True)

civil_num = pd.DataFrame(outcomes[mask_civil].link_uid.value_counts())
civil_num.rename(columns={'link_uid':'civil_ct'},inplace=True)
civil_num.index.rename('link_uid',inplace=True)

abuse_num = pd.DataFrame(outcomes[mask_abuse].link_uid.value_counts())
abuse_num.rename(columns={'link_uid':'abuse_ct'},inplace=True)
abuse_num.index.rename('link_uid',inplace=True)

corrupt_num = pd.DataFrame(outcomes[mask_corrupt].link_uid.value_counts())
corrupt_num.rename(columns={'link_uid':'corrupt_ct'},inplace=True)
corrupt_num.index.rename('link_uid',inplace=True)

# Create a big dataframe with the total complaints, as well as counts for each subtype
comp_list = [oper_num,subs_num,civil_num,abuse_num,corrupt_num]
comp_type = pd.concat(comp_list,axis=1)
# Fill NAs with zeros since they mean no recorded complaints of that type
comp_type = comp_type.fillna(0)
comp_type.index.rename('link_uid',inplace=True)

comp_total = pd.concat([num_complaints,comp_type],axis=1)
comp_total.index.rename('link_uid',inplace=True)



### Make a new roster of officers that includes all counts of complaints and subtypes

In [ ]:
# Join the officers df with the awardsinfo
off_full2 = off_full.join(comp_total,on='link_uid',how='left')
off_full2.set_index('link_uid',inplace=True)

# Drop unnecessary columns and pickle it
off_full2.drop(['start_date','org_hire_date','resignation_date','current_status'],axis=1,inplace=True)
off_full2.to_pickle('officers_aw_comp')

In [ ]:
# Join the complaints df with the officers information
comp_officers = off_full.join(comp1_fulldet.set_index('link_uid'),on='link_uid',how='outer')

# Get rid of officers who don't have complaints (will not have a CV value because they will 
# not have records in the 'outcome' database
comp_officers= comp_officers[pd.notnull(comp_officers['cv'])]


In [ ]:
# Engineer relevant features about age/duty time when complaint occurred, and drop features with high NA
comp_officers['age_off_comp']=(comp_officers['incident_dt']-comp_officers['officer_birthyr'])
comp_officers['time_on_force']=(comp_officers['incident_dt']-comp_officers['appointed_date'])
comp_officers['incident_month']= comp_officers['incident_dt'].dt.month
comp_officers.drop(['resignation_date','current_status'],axis=1,inplace=True)

comp_officers.to_pickle('complaints_officers_final')

In [ ]:
# Join the complaints df with the subtypes of complaints each officer earned
comp_officers_full = comp_officers_upd.join(comp_type,on='link_uid',how='lef`t')

In [ ]:
# Count the number of co-indicated officers for each complaint and make a dataframe
numdef_cv1=pd.DataFrame(comp_officers_full.cr_id.value_counts())
numdef_cv1.columns=['num_coindicated']
numdef_cv1.index.name='cr_id'

comp1_officers = comp_officers_full.join(numdef_cv1,on='cr_id',how='left')

,num_awards,officer_birthyr,officer_race,officer_gender,appointed_date,current_unit,cleaned_rannk,num_complaints,operations_ct,substance_ct,civil_ct,abuse_ct,corrupt_ct
link_uid,,,,,,,,,,,,,
11117.0,387.0,1975-01-01,WHITE,MALE,2004-11-29,8.0,POLICE OFFICER,32.0,5.0,0.0,17.0,10.0,0.0
3549.0,360.0,1982-01-01,WHITE,MALE,2007-08-27,5.0,POLICE OFFICER,23.0,6.0,0.0,14.0,3.0,0.0
6765.0,323.0,1979-01-01,WHITE,MALE,2002-03-25,4.0,SERGEANT,25.0,5.0,0.0,13.0,7.0,0.0
22701.0,322.0,1975-01-01,WHITE,MALE,1999-09-13,4.0,POLICE OFFICER,35.0,7.0,0.0,18.0,8.0,1.0
21124.0,319.0,1978-01-01,WHITE,MALE,2002-04-29,4.0,SERGEANT,56.0,11.0,0.0,31.0,12.0,1.0


## Getting complaint details for 2000-2018 (CV2 and CV3)

In [ ]:
# Convert date details about complaints into datetime features
compdet_cv2['incident_date']=pd.to_datetime(compdet_cv2['incident_date'])
compdet_cv2['complaint_date']=pd.to_datetime(compdet_cv2['complaint_date'])
compdet_cv2['closed_date']=pd.to_datetime(compdet_cv2['closed_date'])

In [ ]:
# Engineer features that indicate time between complaint and incident, and duration of investigation
compdet_cv2['time2complain']=(compdet_cv2.complaint_date-compdet_cv2.incident_date)
compdet_cv2['time2close']=(compdet_cv2.closed_date-compdet_cv2.complaint_date)


In [24]:
# Count the number of complainants for each complaint, and join 
# it with the total dataframe for CV23
numciv_comp23 = pd.DataFrame(comp23_civ.cr_id.value_counts())
numciv_comp23.columns=['numcomplainants']
numciv_comp23.index.name='cr_id'


In [25]:
# Get mode of race, gender and birthday for complaints where there were multiple complainants
comp23_civ.dropna(inplace=True)
comp23_modes = comp23_civ.groupby('cr_id').agg(lambda x:x.value_counts().index[0] )
comp23_modes.drop('cv',axis=1,inplace=True)

In [26]:
# Drop unnecessary columns from the witness dataframe
wit_cv23_trim = wit_cv23.drop(['row_id','complaints_witnesses_2000_2016_2016_11_id','uid','row_id', 'complaint_date','race','gender','birth_year','complainant_subtype'],axis=1)

# Count the number of civilian witness for each complaint and make a dataframe
numwit_cv23=pd.DataFrame(wit_cv23_trim.cr_id.value_counts())
numwit_cv23.columns=['numwitnesses']
numwit_cv23.index.name='cr_id'

In [27]:
# Join demographic complainant data from CV2and3 with the outcomes df, and the number of witnesses associated
comp23_out = comp23_modes.join(outcomes_cv23.set_index('cr_id'),on='cr_id',how='right')
comp23_out = comp23_out.join(numciv_comp23,on='cr_id',how='left')
comp23_out = comp23_out.join(numwit_cv23,on='cr_id',how='left')
comp23_out.numwitnesses.fillna(0,inplace=True)



In [ ]:
# Convert date details about complaints into datetime features
compdet_cv23['incident_date']=pd.to_datetime(compdet_cv23['incident_date'])
compdet_cv23['complaint_date']=pd.to_datetime(compdet_cv23['complaint_date'])
compdet_cv23['closed_date']=pd.to_datetime(compdet_cv23['closed_date'])

# Engineer features that indicate time between complaint and incident, and duration of investigation
compdet_cv23['time2complain']=(compdet_cv23.complaint_date-compdet_cv23.incident_date)
compdet_cv23['time2close']=(compdet_cv23.closed_date-compdet_cv23.complaint_date)
compdet_cv23['incident_month']= compdet_cv23['incident_date'].dt.month


In [ ]:
# Drop the beat - although this is likely relevant, we don't have data before 2000; drop CV to avoid merge conflict
compdet_cv23.drop('beat',axis=1,inplace=True)
compdet_cv23.drop('cv',axis=1,inplace=True)

In [39]:
# Join the df with complainant, outcome, and #witnesses with details about time of complaint
comp23_fulldet = compdet_cv23.join(comp23_out,on='cr_id',how='left')

In [ ]:
# Drop complaints with no record of the category or outcome of complaint (this loses 40k complaints)
comp23_fulldet.dropna(inplace=True,subset=['final_finding'])

In [ ]:
# Set the value for 'officer-filed' to equal true if the complaint ID is present in the list
# of complaints filed/corroborated by officers
comp23_fulldet['officer_filed'][comp23_fulldet['cr_id'].isin(comp23_off)]=True

In [41]:
# Join the complaints df for CV23 with the officers information
comp23_officers = comp23_fulldet.join(officers_df,on='link_uid',how='left')


In [42]:
# Engineer relevant features about age/duty time when complaint occurred, and drop features with high NA
comp23_officers['age_off_comp']=(comp23_officers['incident_date']-comp23_officers['officer_birthyr'])
comp23_officers['time_on_force']=(comp23_officers['incident_date']-comp23_officers['appointed_date'])

# Sum complainants with witnesses, since pre-2000 data lumped them together
comp23_officers['numwitnesses']=comp23_officers['numwitnesses']+comp23_officers['numcomplainants']

In [43]:
# Count the number of co-indicated officers for each complaint and make a dataframe
numdef_cv23=pd.DataFrame(comp23_officers.cr_id.value_counts())
numdef_cv23.columns=['num_coindicated']
numdef_cv23.index.name='cr_id'

comp23_officers = comp23_officers.join(numdef_cv23,on='cr_id',how='left')


## Joining and cleaning full dataset

In [ ]:
# Rename columns to match in both dfs
comp1_officers.rename(columns={'closed_dt': 'closed_date','complaint_dt': 'complaint_date','incident_dt': 'incident_date'},inplace=True)
comp1_officers.drop('officer_or_non',axis=1,inplace=True)
comp1_officers.to_pickle('comp1_ready2model')
comp23_officers.to_pickle('comp23_ready2model')

In [ ]:
comp1_officers = pd.read_pickle('comp1_ready2model')
comp23_officers = pd.read_pickle('comp23_ready2model')

In [ ]:
# Join dfs from C1 and CV23
all_complaints = pd.concat([comp1_officers,comp23_officers])

In [ ]:
# Convert timedeltas to integers to prepare for EDA and modeling
all_complaints['age_off_comp']=all_complaints['age_off_comp'].dt.days
all_complaints['time2close']=all_complaints['time2close'].dt.days
all_complaints['time2complain']=all_complaints['time2complain'].dt.days
all_complaints['time_on_force']=all_complaints['time_on_force'].dt.days

In [ ]:
# Convert final outcome to all lower case to make combining outcomes easier
all_complaints['final_outcome']= all_complaints['final_outcome'].str.lower()

### Recode discipline to be binary

In [ ]:
# Simplify outcomes by binning

# Short suspension - 1 to 7 days
all_complaints.final_outcome.replace({'1 day suspension': 'short suspension',
                                      '2 day suspension': 'short suspension',
                                      '3 day suspension': 'short suspension',
                                      '4 day suspension': 'short suspension',
                                      '5 day suspension': 'short suspension',
                                      '6 day suspension': 'short suspension',
                                      '7 day suspension': 'short suspension'},inplace=True)

# Medium suspension - 8 to 29 days
all_complaints.final_outcome.replace({'8 day suspension': 'medium suspension',
                                      '9 day suspension': 'medium suspension',
                                      '10 day suspension': 'medium suspension',
                                      '11 day suspension': 'medium suspension',
                                      '12 day suspension': 'medium suspension',
                                      '13 day suspension': 'medium suspension',
                                      '14 day suspension': 'medium suspension',
                                      '15 day suspension': 'medium suspension',
                                      '16 day suspension': 'medium suspension',
                                      '17 day suspension': 'medium suspension',
                                      '18 day suspension': 'medium suspension',
                                      '20 day suspension': 'medium suspension',
                                      '21 day suspension': 'medium suspension',
                                      '22 day suspension': 'medium suspension',
                                      '23 day suspension': 'medium suspension',
                                      '24 day suspension': 'medium suspension',
                                      '25 day suspension': 'medium suspension',
                                      '27 day suspension': 'medium suspension',
                                     '28 day suspension': 'medium suspension',
                                     '29 day suspension': 'medium suspension'},inplace=True)


# Long suspension - 30 or more days (some suspensions are very long - possibly some kind of retirement gift)
all_complaints.final_outcome.replace({'suspen\'d indefinit\'y': 'long suspension',
                                      'suspended for 180 days': 'long suspension',
                                      '540': 'long suspension',
                                      '900 day suspension': 'long suspension',
                                      '365': 'long suspension',
                                      '326': 'long suspension',
                                      '40': 'long suspension',
                                      '35': 'long suspension',
                                      '365 day suspension': 'long suspension',
                                      '270 day suspension': 'long suspension',
                                      '180 day suspension': 'long suspension',
                                      '150 day suspension': 'long suspension',
                                      '120 day suspension': 'long suspension',
                                      '90 day suspension': 'long suspension',
                                      '75 day suspension': 'long suspension',
                                      '60 day suspension': 'long suspension',
                                      '45 day suspension': 'long suspension',
                                      '40 day suspension': 'long suspension',
                                      '35 day suspension': 'long suspension',
                                      '31 day suspension': 'long suspension',
                                      '30 day suspension': 'long suspension',
                                      'suspended ovr 30 day': 'long suspension',
                                      'suspended over 30 days': 'long suspension'},inplace=True)

# Combine different encodings -  in case of numeric encodings this required investigated the actual 
# records to find out the penalty
all_complaints.final_outcome.replace({'**penalty not served': 'penalty not served'},inplace=True)
all_complaints.final_outcome.replace({'resigned -not served': 'resigned',
                                      '99': 'resigned'
                                     },inplace=True)
all_complaints.final_outcome.replace({'reinstated by police board': 'reinstated',
                                      'reinstated by court action': 'reinstated',
                                      'reinstated police bd': 'reinstated',
                                      'reinstated court act': 'reinstated',
                                     },inplace=True)
all_complaints.final_outcome.replace({'999': 'unknown'},inplace=True)

# Lump any case where officer was fired together
all_complaints.final_outcome.replace({'admin. termination': 'termination',
                                      'administrative termination': 'termination',
                                      'separation': 'termination',
                                      'separated other case': 'termination'
                                     },inplace=True)


In [ ]:
# Create a binary column - disciplined or not. Set it to not
all_complaints['discipline']=0

# Set discipline column to 1 any case where discipline occurred
all_complaints.discipline.where((all_complaints['final_outcome']=='resigned')|
                                (all_complaints['final_outcome']=='no action taken')|
                                (all_complaints['final_outcome']=='sustained-no penalty')|
                                (all_complaints['final_outcome']=='reinstated')|
                                (all_complaints['final_outcome']=='unknown')|
                                (all_complaints['final_outcome']=='penalty not served')|
                                (all_complaints['final_outcome']=='violation noted')
                                ,1,inplace=True)

# Numbers approximately matched those on CPDP database (247150 complaints, 7% disciplined)
all_complaints.discipline.value_counts()

In [3]:
all_complaints=pd.read_pickle('fulldata_model')

In [10]:
all_complaints.to_pickle('fulldata_model')

In [9]:
# EDA - look at quantitative metrics and see which are very different between 1 and 0 discipline
all_complaints.groupby('discipline').mean()

,abuse_ct,age_off_comp,civil_ct,corrupt_ct,cv,incident_month,num_awards,num_coindicated,num_complaints,numwitnesses,...,substance_ct,time2close,time2complain,time_on_force,not_operations,injury,administrator,copa_style,search,reform
discipline,,,,,,,,,,,,,,,,,,,,,
0,8.241770,14099.463663,6.565385,1.400223,1.511921,6.465638,39.702374,3.311141,23.842463,1.417103,...,0.069982,235.633488,28.496847,3646.593105,0.691542,0.188653,0.221564,0.546683,0.203297,0.230064
1,6.302192,14842.888939,3.674030,1.708881,1.494579,6.465423,23.914387,4.049829,19.908994,1.719027,...,0.120461,669.617895,30.941127,4205.246601,0.472558,0.121679,0.185383,0.207123,0.013033,0.179372


### Encoding complex columns to be binary (reform) and simplifying cop jargon

In [8]:
# Create a column called 'reform' to specify which era of police reform we are in
all_complaints['reform']=0
ipra = pd.Timestamp(2007, 7, 1)
copa = pd.Timestamp(2017, 9, 1)
all_complaints['reform']= all_complaints.reform.where((all_complaints['complaint_date']<=ipra),1)
all_complaints.reform.value_counts()

0    181012
1     52916
Name: reform, dtype: int64

In [ ]:
# Some previous years use wonky complaint code numbering - recode these according to the normal system
all_complaints.number_complaint_code.replace({'253': '18','251': '18','249': '18','085': '18',
                                              '001':'05','002':'05','005':'05','006':'05','007':'05',
                                              '008':'05','009':'05','010':'05','011':'05',
                                              '014':'20','035':'20','093':'20','094':'20',
                                              '022':'05','023':'05','025':'05','026':'05','028':'05','029':'05',
                                              '030':'05','031':'05','043':'05','044':'05','051':'05','058':'05',
                                              '065':'05','101':'05','109':'05','110':'05',
                                              '112':'05','114':'05','117':'05','118':'05',
                                             '113':'01'},inplace=True)

In [ ]:
#Impute race and gender for complaints that are brought automatically by the government

all_complaints.gender.where((all_complaints['complaint_category']!="INDEBTEDNESS TO CITY"),'GOVT', inplace=True)
all_complaints.race.where((all_complaints['complaint_category']!="INDEBTEDNESS TO CITY"),'GOVT', inplace=True)

all_complaints.gender.where((all_complaints['complaint_category']!="STATE CIVIL SUIT"),'GOVT', inplace=True)
all_complaints.race.where((all_complaints['complaint_category']!="STATE CIVIL SUIT"),'GOVT', inplace=True)

In [ ]:
# Create a column to sort complaint types by operations vs not operations
all_complaints['not_operations']=1
all_complaints.not_operations.where((all_complaints['number_complaint_code']!="10"),0, inplace=True)
all_complaints.not_operations.where((all_complaints['number_complaint_code']!="12"),0, inplace=True)
all_complaints.not_operations.where((all_complaints['number_complaint_code']!="17"),0, inplace=True)
all_complaints.not_operations.value_counts()

In [ ]:
# Create a column to sort complaints that purportedly injured the victim or not
all_complaints['injury']=0
all_complaints.injury.where((all_complaints['number_complaint_code']!="18"),1, inplace=True)
all_complaints.injury.where((all_complaints['number_complaint_code']!="19"),1, inplace=True)
all_complaints.injury.where((all_complaints['number_complaint_code']!="20"),1, inplace=True)
all_complaints.injury.where((all_complaints['complaint_code']!="10W"),1, inplace=True)
all_complaints.injury.where((all_complaints['complaint_code']!="08A"),1, inplace=True)
all_complaints.injury.where((all_complaints['complaint_code']!="08B"),1, inplace=True)
all_complaints.injury.where((all_complaints['complaint_code']!="08C"),1, inplace=True)
all_complaints.injury.where((all_complaints['complaint_code']!="05K"),1, inplace=True)
all_complaints.injury.where((all_complaints['complaint_code']!="05G"),1, inplace=True)
all_complaints.injury.where((all_complaints['complaint_code']!="05E"),1, inplace=True)
all_complaints.injury.where((all_complaints['complaint_code']!="05C"),1, inplace=True)
all_complaints.injury.where((all_complaints['complaint_code']!="05A"),1, inplace=True)
all_complaints.injury.where((all_complaints['complaint_code']!="02C"),1, inplace=True)
all_complaints.injury.where((all_complaints['complaint_code']!="04H"),1, inplace=True)
all_complaints.injury.where((all_complaints['complaint_code']!="02D"),1, inplace=True)

In [106]:
# Create a column to sort rank by administrator or not
all_complaints['administrator']=1
all_complaints.administrator.where((all_complaints['cleaned_rannk']!="POLICE OFFICER"),0, inplace=True)
all_complaints.administrator.where((all_complaints['cleaned_rannk']!="DETECTIVE"),0, inplace=True)

In [118]:
# Create a column to capture COPA-relevant complaints
all_complaints['copa_style']=0
all_complaints.copa_style.where((all_complaints['number_complaint_code']!="03"),1, inplace=True)
all_complaints.copa_style.where((all_complaints['number_complaint_code']!="01"),1, inplace=True)
all_complaints.copa_style.where((all_complaints['number_complaint_code']!="05"),1, inplace=True)
all_complaints.copa_style.where((all_complaints['number_complaint_code']!="18"),1, inplace=True)
all_complaints.copa_style.where((all_complaints['number_complaint_code']!="19"),1, inplace=True)
all_complaints.copa_style.where((all_complaints['number_complaint_code']!="20"),1, inplace=True)
all_complaints.copa_style.where((all_complaints['number_complaint_code']!="21"),1, inplace=True)
all_complaints.copa_style.where((all_complaints['complaint_code']!="04C"),1, inplace=True)
all_complaints.copa_style.where((all_complaints['complaint_code']!="04G"),1, inplace=True)
all_complaints.copa_style.where((all_complaints['complaint_code']!="04H"),1, inplace=True)
all_complaints.copa_style.where((all_complaints['complaint_code']!="12F"),1, inplace=True)

In [119]:
# Create a column to capture 'search' complaints
all_complaints['search']=0
all_complaints.search.where((all_complaints['number_complaint_code']!="03"),1, inplace=True)


In [ ]:
# Filter out all unmappable complaint codes
unk_codes = ['021','042','116','083','020','181','11','13','14']
all_complaints = all_complaints[~all_complaints['number_complaint_code'].isin(unk_codes)]

# Make sure only interpretable complaints are present
all_complaints.number_complaint_code.value_counts()

In [ ]:
all_complaints[['complaint_category','copa_style']].sample(20)

### Restrict the timeframe to 2000 to 2015

Since complaints that result in discipline frequently take longer than a year to reach a sentence, I will exclude data for complaints filed after 2015 ended. That way I will not bias my data in favor of no-discipline findings.

I will also exclude data from before 2001, to make it more symmetrical around when IPRA came into effect.

In [81]:
y2000 = pd.Timestamp(2001, 1, 1)
y2016 = pd.Timestamp(2016,1,1)
complaint00to16 = all_complaints[(all_complaints['complaint_date']>y2000) & (all_complaints['complaint_date']<y2016)]

In [82]:
complaint00to16.to_pickle('complaints_timerestricted')

In [78]:
complaint00to16.groupby('discipline').mean()

,abuse_ct,age_off_comp,civil_ct,corrupt_ct,cv,incident_month,num_awards,num_coindicated,num_complaints,numwitnesses,...,substance_ct,time2close,time2complain,time_on_force,not_operations,injury,administrator,copa_style,search,reform
discipline,,,,,,,,,,,,,,,,,,,,,
0,8.203825,14105.677630,7.688699,1.155646,1.916308,6.291735,55.060718,3.227968,24.661949,2.201971,...,0.057101,276.607897,35.651941,3577.234593,0.682450,0.148854,0.206632,0.566679,0.261827,0.407591
1,6.860143,15019.772605,4.532239,1.471396,2.081136,6.344187,35.918057,2.920614,21.216362,3.367958,...,0.104590,774.579042,51.490716,4169.235429,0.488563,0.124865,0.179225,0.265339,0.011168,0.377018


In [11]:
officers_df = pd.read_pickle('officers_aw_comp')
officers_df.head()

,num_awards,officer_birthyr,officer_race,officer_gender,appointed_date,current_unit,cleaned_rannk,num_complaints,operations_ct,substance_ct,civil_ct,abuse_ct,corrupt_ct
link_uid,,,,,,,,,,,,,
11117.0,387.0,1975-01-01,WHITE,MALE,2004-11-29,8.0,POLICE OFFICER,32.0,5.0,0.0,17.0,10.0,0.0
3549.0,360.0,1982-01-01,WHITE,MALE,2007-08-27,5.0,POLICE OFFICER,23.0,6.0,0.0,14.0,3.0,0.0
6765.0,323.0,1979-01-01,WHITE,MALE,2002-03-25,4.0,SERGEANT,25.0,5.0,0.0,13.0,7.0,0.0
22701.0,322.0,1975-01-01,WHITE,MALE,1999-09-13,4.0,POLICE OFFICER,35.0,7.0,0.0,18.0,8.0,1.0
21124.0,319.0,1978-01-01,WHITE,MALE,2002-04-29,4.0,SERGEANT,56.0,11.0,0.0,31.0,12.0,1.0


### Create a final dataset of complaints which have not yet received a sentence, or whose sentence was redacted in the FOIA

#### All of the following cells simply go through the entire cleaning process from start to finish, now looking at just the unknown data

In [52]:
test1 = comp23_officers[comp23_officers['final_outcome']=="Unknown"]

In [ ]:
# Create a column called 'reform' to specify which era of police reform we are in
test1['reform']=0
ipra = pd.Timestamp(2007, 7, 1)
copa = pd.Timestamp(2017, 9, 1)
test1['reform']= test1.reform.where((test1['complaint_date']<=ipra),1)

In [ ]:
# Some previous years use wonky complaint code numbering - recode these according to the normal system
test1.number_complaint_code.replace({'253': '18','251': '18','249': '18','085': '18',
                                              '001':'05','002':'05','005':'05','006':'05','007':'05',
                                              '008':'05','009':'05','010':'05','011':'05',
                                              '014':'20','035':'20','093':'20','094':'20',
                                              '022':'05','023':'05','025':'05','026':'05','028':'05','029':'05',
                                              '030':'05','031':'05','043':'05','044':'05','051':'05','058':'05',
                                              '065':'05','101':'05','109':'05','110':'05',
                                              '112':'05','114':'05','117':'05','118':'05',
                                             '113':'01'},inplace=True)

In [ ]:
# Create a column to sort complaint types by operations vs not operations
test1['not_operations']=1
test1.not_operations.where((test1['number_complaint_code']!="10"),0, inplace=True)
test1.not_operations.where((test1['number_complaint_code']!="12"),0, inplace=True)
test1.not_operations.where((test1['number_complaint_code']!="17"),0, inplace=True)
test1.not_operations.value_counts()

In [ ]:
test1['injury']=0
test1.injury.where((test1['number_complaint_code']!="18"),1, inplace=True)
test1.injury.where((test1['number_complaint_code']!="19"),1, inplace=True)
test1.injury.where((test1['number_complaint_code']!="20"),1, inplace=True)
test1.injury.where((test1['complaint_code']!="10W"),1, inplace=True)
test1.injury.where((test1['complaint_code']!="08A"),1, inplace=True)
test1.injury.where((test1['complaint_code']!="08B"),1, inplace=True)
test1.injury.where((test1['complaint_code']!="08C"),1, inplace=True)
test1.injury.where((test1['complaint_code']!="05K"),1, inplace=True)
test1.injury.where((test1['complaint_code']!="05G"),1, inplace=True)
test1.injury.where((test1['complaint_code']!="05E"),1, inplace=True)
test1.injury.where((test1['complaint_code']!="05C"),1, inplace=True)
test1.injury.where((test1['complaint_code']!="05A"),1, inplace=True)
test1.injury.where((test1['complaint_code']!="02C"),1, inplace=True)
test1.injury.where((test1['complaint_code']!="04H"),1, inplace=True)
test1.injury.where((test1['complaint_code']!="02D"),1, inplace=True)
test1.injury.value_counts()

In [ ]:
# Create a column to sort rank by administrator or not
test1['administrator']=1
test1.administrator.where((test1['cleaned_rannk']!="POLICE OFFICER"),0, inplace=True)
test1.administrator.where((test1['cleaned_rannk']!="DETECTIVE"),0, inplace=True)
test1.administrator.value_counts()

In [ ]:
# Create a column to capture COPA-relevant complaints
test1['copa_style']=0
test1.copa_style.where((test1['number_complaint_code']!="03"),1, inplace=True)
test1.copa_style.where((test1['number_complaint_code']!="01"),1, inplace=True)
test1.copa_style.where((test1['number_complaint_code']!="05"),1, inplace=True)
test1.copa_style.where((test1['number_complaint_code']!="18"),1, inplace=True)
test1.copa_style.where((test1['number_complaint_code']!="19"),1, inplace=True)
test1.copa_style.where((test1['number_complaint_code']!="20"),1, inplace=True)
test1.copa_style.where((test1['number_complaint_code']!="21"),1, inplace=True)
test1.copa_style.where((test1['complaint_code']!="04C"),1, inplace=True)
test1.copa_style.where((test1['complaint_code']!="04G"),1, inplace=True)
test1.copa_style.where((test1['complaint_code']!="04H"),1, inplace=True)
test1.copa_style.where((test1['complaint_code']!="12F"),1, inplace=True)

In [ ]:
# Create a column to capture 'search' complaints
test1['search']=0
test1.search.where((test1['number_complaint_code']!="03"),1, inplace=True)


In [ ]:
# Filter out all unmappable complaint codes
unk_codes = ['021','042','116','083','020','181','11','13','14']
test1 = test1[~test1['number_complaint_code'].isin(unk_codes)]

# Make sure only interpretable complaints are present
test1.number_complaint_code.value_counts()

In [66]:
test1.to_pickle('unknown_complaints')